## Set-up and Data Loading

In [ ]:
from sentence_transformers import SentenceTransformer, util, models
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from umap import UMAP
from hdbscan import HDBSCAN
import os
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from bertopic.representation import MaximalMarginalRelevance
from sklearn.manifold import TSNE
import plotly.express as px
from tqdm.auto import tqdm
import seaborn as sns
import gensim
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
# from octis.evaluation_metrics.diversity_metrics import TopicDiversity

In [ ]:
path_data = '..\\data_structured'

In [ ]:
path_reports = '..\\data\\reports'

In [ ]:
sample = os.listdir(path_reports)
# converting to panda series because I find it easier to manipulate
company_list = [word.split('.')[0] for word in sample]
company_list = pd.Series(company_list)
publisher_list = company_list.str.replace('-',' ') #so make it two words
publisher_list = publisher_list.str.replace('ford motor', 'ford')
publisher_list = publisher_list.str.replace('p&g', 'procter')
publisher_list

In [ ]:
names_list = publisher_list.tolist()

In [ ]:
names_list.append('p&g')

In [ ]:
names_list

In [ ]:
df_report = pd.read_csv(os.path.join(path_data,'report_sentences.csv'))
df_pdf = pd.read_csv(os.path.join(path_data,'article_sentences_pdf.csv'))
df_gnews = pd.read_csv(os.path.join(path_data,'article_sentences_gnews.csv'))
df_article = pd.concat([df_pdf,df_gnews])

In [ ]:
df_article.shape

In [ ]:
df_report.shape

In [ ]:
df_article.drop_duplicates(subset = ['sentence'], inplace = True)
df_article.reset_index(inplace = True, drop = True)

In [ ]:
## part of the extra step below
#df_combined = pd.concat([df_article, df_report])

In [ ]:
sent_embedder = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

In [ ]:
df_rep = df_report.copy()
df_art = df_article.copy()

In [ ]:
# function for generating embeddings (useful for testing out later):
def gen_embeddings(df,model):
    sentences = df['sentence'].tolist()
    return model.encode(sentences)

In [ ]:
%%time
# creating embeddings for the report sentences and storing them in a new column
df_rep['embeddings'] = list(gen_embeddings(df_rep,sent_embedder))

In [ ]:
%%time
df_art['embeddings'] = list(gen_embeddings(df_art, sent_embedder))

- now we test out BERTopic 
- gonna try it with the overlapped sentence embeddings, non-overlapped?, and climateBERT + potentially FINbert
- then we try to manipulate the different parameters - tuning?

In [ ]:
df_rep.to_pickle(os.path.join(path_data,'rep.pkl'))
df_art.to_pickle(os.path.join(path_data,'art.pkl'))
# df_art = pd.read_pickle(os.path.join(path_data,'art.pkl'))
# df_rep = pd.read_pickle(os.path.join(path_data,'rep.pkl'))

In [ ]:
# combining the two datasets in preparation for the clustering
df_comb = pd.concat([df_art, df_rep])

In [ ]:
df_comb.shape

In [ ]:
df_comb['char_length'] = df_comb['sentence'].apply(lambda x: len(x))

In [ ]:
df_comb = df_comb[(df_comb['char_length'] > 20)]
df_comb.shape

In [ ]:
df_comb.drop_duplicates(subset = ['sentence'], inplace = True)
df_comb.shape

In [ ]:
import re
# Escaping the names_list for safe inclusion in the regular expression pattern
escaped_names = [re.escape(name) for name in names_list]

# Constructing the modified regular expression pattern
pattern = r"\b(?:{})(?:'s)?\b".format("|".join(escaped_names))

# Replace company names with 'company' using the regular expression pattern
df_comb['anon_sentence'] = df_comb['sentence'].str.replace(pattern, 'the company', case=False, regex=True)

# Print the modified dataframe
df_comb['anon_sentence'].values[200:260]

In [ ]:
df_comb.reset_index(inplace = True, drop = True)

In [ ]:
# Load the sentence embeddings and doc types from the dataframe
# embeddings = np.array(df_comb["embeddings"].tolist())

In [ ]:
# creating new embeddings based on the 'anonymized' sentences (without company names) - let's see how BERTopic handles it now
df_comb['anon_embeddings'] = list(gen_embeddings(df_comb, sent_embedder))

## Visualizations

In [ ]:
# Create a histogram of the word count
plt.figure(figsize=(10,5))
plt.hist(df_comb['word count'], bins=30, color='skyblue')
plt.title('Word Count Distribution')
plt.xlabel('Word Count')
plt.ylabel('Frequency')
plt.savefig('word_count_hist.png', dpi=300)

plt.show()

In [ ]:
# # Group the data by company, doc_type, and count the number of sentences
# df_by_company_doc_type = df_comb.groupby(['company', 'doc_type'])['sentence'].count().reset_index()

# # Pivot the dataframe to create separate columns for each doc_type
# df_pivot = df_by_company_doc_type.pivot(index='company', columns='doc_type', values='sentence')

# Create a line chart of the number of sentences by company and doc_type
plt.figure(figsize=(20,5));
df_pivot.plot.line(color=['red', 'blue'], marker='o', markersize=5)
plt.title('Number of Sentences by Company and Doc Type')
plt.xlabel('Company')
plt.ylabel('Number of Sentences')
plt.legend(title='Doc Type')

# Save the plot as a PNG file
plt.savefig('sentences_by_company_and_doc_type.png', dpi=300)
plt.show()

In [ ]:
umap2d = UMAP(n_components = 2, init = 'random', random_state = 0)
# umap3d = UMAP(n_components = 3, init = 'random', random_state = 0)

proj_2d = umap2d.fit_transform(embeddings)
# proj_3d = umap3d.fit_transform(embeddings)

In [ ]:
fig2d = px.scatter(
    proj_2d, x = 0, y = 1, 
    color = df_comb.doc_type, labels = {'color': 'doc_type'},
    width = 800, height = 500
)
# fig3d = px.scatter_3d(
#     proj_3d, x = 0, y = 1, z = 2,
#     color = df_comb.doc_type, labels = {'color': 'doc_type'}
# )

fig2d.show()
# fig3d.show()

In [ ]:
tsne2d = TSNE(n_components = 2, random_state = 0)
tsne3d = TSNE(n_components = 3, random_state = 0)

proj_2d_tsne = tsne2d.fit_transform(embeddings)
proj_3d_tsne = tsne3d.fit_transform(embeddings)


In [ ]:
fig2d_tsne = px.scatter(
    proj_2d_tsne, x = 0, y = 1, 
    color = df_comb.doc_type, labels = {'color': 'doc_type'}
)

# fig3d_tsne = px.scatter_3d(
#     proj_3d_tsne, x = 0, y = 1, z = 2,
#     color = df_comb.doc_type, labels = {'color': 'doc_type'}
# )

fig2d_tsne.show()
# fig3d_tsne.show()

Based on the above visualizations, we can see that unlike Boelders, we do not require to overlap the sentence embeddings over each other, as there are not large semantic differences between the news articles and the reports. The jargon and language seems to be mostly the same, as such we can just use the original embeddings.

## BERTopic Set-up

### Testing UMAP parameters

In [ ]:
colors = df_comb['doc_type']
c_map = {
    'news': '#FAFF00',
    'report': '#1C17FF'
}
colors = [c_map[x] for x in colors]
len(colors)

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(21, 10))
nns = [5,10, 15, 30, 50, 100]
#2, 3, 4
i, j = 0, 0
for n_neighbors in tqdm(nns):
    fit = UMAP(n_neighbors=n_neighbors, random_state = 0)
    u = fit.fit_transform(embeddings)
    sns.scatterplot(x=u[:,0], y=u[:,1], ax=ax[j, i])
    ax[j, i].set_title(f'n={n_neighbors}')
    if i < 2: i += 1
    else: i = 0; j += 1

In [ ]:
fig.savefig('full_figure.png')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 14))
nns = [3,4]
#2, 3, 4
i, j = 0, 0
for n_neighbors in tqdm(nns):
    fit = UMAP(n_neighbors=n_neighbors, random_state = 0)
    u = fit.fit_transform(embeddings)
    sns.scatterplot(x=u[:,0], y=u[:,1], c=colors, ax=ax[i])
    ax[i].set_title(f'n={n_neighbors}')
    i +=1

In [ ]:
fit = UMAP(n_neighbors=10, n_components=3, min_dist=0.0, init = 'random', metric = 'cosine', random_state = 0)
u = fit.fit_transform(embeddings)

# the lowest min distance finally creates some separation - what does increasing it do?

In [ ]:
fig = px.scatter_3d(
    x=u[:,0], y=u[:,1], z=u[:,2],
    color=df_comb.doc_type,
    labels = {'color': 'doc_type'}
)

fig.show()

In [ ]:
# 2D representation
fit2d = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, init = 'random', metric = 'cosine', random_state = 0)
u2d = fit2d.fit_transform(embeddings)

fig2d = px.scatter(
    x=u2d[:,0], y=u2d[:,1],
    color=df_comb.doc_type,
    labels = {'color': 'doc_type'},
    width = 800, height = 500
)

fig2d.show()

In [ ]:
# 2D representation
fit2d = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, init = 'random', metric = 'cosine', random_state = 0)
u2d = fit2d.fit_transform(embeddings)

fig2d = px.scatter(
    x=u2d[:,0], y=u2d[:,1],
    color=df_comb.doc_type,
    labels = {'color': 'doc_type'},
    width = 800, height = 500
)

fig2d.show()

### Clustering with HDBSCAN

In [ ]:
clusterer = HDBSCAN(min_cluster_size = 400)
clusterer.fit(u)

In [ ]:
clusterer.condensed_tree_.plot(select_clusters=True)

In [ ]:
clusterer = HDBSCAN(min_cluster_size = 400, min_samples = 50, metric = 'euclidean')
clusterer.fit(u)
clusterer.condensed_tree_.plot(select_clusters=True)

## Running BERTopic

In [ ]:
docs = df_comb['anon_sentence']

In [ ]:
embeddings = np.array(df_comb["anon_embeddings"].tolist())

In [ ]:
# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0,init = 'random', metric = 'cosine', random_state = 0).fit_transform(embeddings)

In [ ]:
# getting stopwords
stopwords = list(stopwords.words('english')) + ['company','coca','cola'] + names_list

In [ ]:
# customizing parts of the BERTopic pipeline
# vectorizer
vectorizer_model = CountVectorizer(stop_words = stopwords)

# umap
umap_model = UMAP(n_neighbors=10, n_components=5, min_dist=0.0, init = 'random', metric = 'cosine', random_state = 0)

# hdbscan
hdbscan_model = HDBSCAN(min_cluster_size = 200, metric='euclidean', prediction_data=True)
# 200 has decent results
# min_samples
# diversity topic words
representation_model = MaximalMarginalRelevance(diversity=0.5)

In [ ]:
topic_model = BERTopic(
    embedding_model = sent_embedder,
    vectorizer_model = vectorizer_model,
    umap_model = umap_model,
    hdbscan_model = hdbscan_model,
    language="english", 
    representation_model = representation_model, #diversify topic words
    calculate_probabilities=True, 
    verbose=True, 
    #nr_topics = 'auto'
    # min_topic_size = 50, 
    # n_gram_range = (1,2)
)
topics, probs = topic_model.fit_transform(docs, embeddings)

In [ ]:
def find_highest_probability(array):
    max_index = np.argmax(array)
    max_prob = array[max_index]
    return max_prob, max_index
find_highest_probability(probs[10])

In [ ]:
topic_model.get_topic_info()[:60]

In [ ]:
topic_model.get_topic_info()[:60]

In [ ]:
topic_model.visualize_barchart(top_n_topics=10, n_words = 10, height = 400)

In [ ]:
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, 
                                hide_document_hover=True, hide_annotations=True)

## Topic Reduction

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(docs)

In [ ]:
topic_model.get_topic(26)

In [ ]:
topic_model.get_representative_docs(26)

In [ ]:
>>> tree = topic_model.get_topic_tree(hierarchical_topics)
>>> print(tree)

# saf stands for sustainabile aviation fuel - together with topic 13 talks about the opportunities for sustainable fuel to reduce mostly emissions - can call it something like sustainable transportation? with dhl

In [ ]:
topic_model.visualize_hierarchical_documents(docs, hierarchical_topics, reduced_embeddings=reduced_embeddings)

In [ ]:
# Now reduction of topics - we will merge them manually based ont he above visualizations
topics_to_merge = [
    [20,33,3,14], #plastic packaging and recycling
    [12,4,7,51,60,24,59,30], #recycling, more focus on circular economy other than just plastic - include 59? (printing), 32 (sustainable materials/sourcing?)
    [58,56,35,21,13], #regenerative agriculture/sustainable agriculture
    [8,23,27], #water, biodiversity, deforestation - could be called nature preservation
    [43,16], #sustainability (tech) innovation
    #[5,28], #climate change
    [1,6,17,22], #sustainable leadership/governance?
    [29,11,25,19], #sustainable fuels/transportations
    [39,40,2,15,53,61,45,47,30,32], #decarbonisation,emission reductions - include 47? (air pollution), 30? (chemical substances)
    [18,34,46]#EV's
] 
topic_model.merge_topics(docs, topics_to_merge)

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.get_representative_docs(25)

## Outlier Reduction

In [ ]:
df_comb = pd.read_pickle(os.path.join(path_data,'comb.pkl'))

In [ ]:
topic_model.save('merged_model')

In [ ]:
topic_model = BERTopic.load('merged_model')

In [ ]:
topics = topic_model.topics_
probs = topic_model.probabilities_

In [ ]:
%%time
#new_topics_probs = topic_model.reduce_outliers(docs, topics, probabilities=probs, strategy="probabilities")
#new_topics_tfidf = topic_model.reduce_outliers(docs, topics, strategy="c-tf-idf", threshold = 0.1)
new_topics_dist = topic_model.reduce_outliers(docs, topics, strategy = "distributions", threshold = 0.08)

In [ ]:
new_topics_embed = topic_model.reduce_outliers(docs, topics, strategy = "embeddings", threshold = 0.23)

### HDBSCAN Probabilities

In [ ]:
topic_model.update_topics(docs, topics=new_topics_probs, vectorizer_model = vectorizer_model, representation_model = representation_model)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, 
                                hide_document_hover=True, hide_annotations=True)

In [ ]:
topic_model.get_topic_info()

### C-TF-IDF similarity

In [ ]:
topic_model.update_topics(docs, topics=new_topics_tfidf, vectorizer_model = vectorizer_model, representation_model = representation_model)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, 
                                hide_document_hover=True, hide_annotations=True)

In [ ]:
topic_model.get_topic_info()

### Using topic distributions

In [ ]:
topic_model.update_topics(docs, topics=new_topics_dist, vectorizer_model = vectorizer_model, representation_model = representation_model) #  representation_model = representation_model
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, 
                                hide_document_hover=True, hide_annotations=True)

In [ ]:
topic_model.get_topic_info()

In [ ]:
# topic_model.set_topic_labels({0: "ESG governance", 1: "Renewable Energy", 2: "Waste Management", 3: "Emission Reduction", 4: "Plastics Recyling", 5: "Nature Preservation", 6:"Sustainable Transportation", 7: "Climate Change", 8: "Sustainable Agriculture", 9: "Stocks",
#                              10: "Electrical Vehicles", 12:"Sustainable Innovation", 14:"No Comment", })

In [ ]:
topics = topic_model.topics_
df_comb['topics'] = topics

In [ ]:
df_comb.to_pickle(os.path.join(path_data,'comb.pkl'))

In [ ]:
df_comb['topics'].value_counts()

In [ ]:
list(df_comb[df_comb['topics'] == 27]['anon_sentence'].values)

In [ ]:
# second merging after taking a look at the reports
topics_to_merge = [
    [26,10], #green cars/car production
    [14,3], #pollution lawsuits included
    
] 
topic_model.merge_topics(docs, topics_to_merge)

In [ ]:
topic_model.get_topic_info()

In [ ]:
from wordcloud import WordCloud

def create_wordcloud(model, topic, save_path = None):
    text = {word: value for word, value in model.get_topic(topic)}
    wc = WordCloud(background_color="white", max_words=1000)
    wc.generate_from_frequencies(text)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    if save_path:
        plt.savefig(os.path.join(save_path, f"wordcloud_{topic}.png"))
    else:
        plt.show()


In [ ]:
sust_topics = [0,1,2,3,4,5,6,7,8,10,11,20,25]
save_path = ".\\visualizations"

for i in sust_topics:
    create_wordcloud(topic_model, i, save_path = save_path)

In [ ]:
text = {word: value for word, value in topic_model.get_topic(0)}

In [ ]:
text

### Using Sentence and Topic Embedding Similarity

In [ ]:
#topic_model.update_topics(docs, topics=new_topics_embed, vectorizer_model = vectorizer_model,representation_model = representation_model)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, 
                                hide_document_hover=False, hide_annotations=True)

In [ ]:
topic_model.get_topic_info()

## Fitting different embedding models

In [ ]:
# mpnet - don't have high hopes for this one
model_mpnet = SentenceTransformer('all-mpnet-base-v2')
mpnet_embeddings = gen_embeddings(len(df_comb), model_mpnet) 

np.save('mpnet_embeddings', mpnet_embeddings)

In [ ]:
len(clim_embeddings)

In [ ]:
topics_clim = topic_model.fit_transform(docs, clim_embeddings)

In [ ]:
topic_model.get_topic_info()

In [ ]:
reduced_clim_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0,init = 'random', metric = 'cosine', random_state = 0).fit_transform(clim_embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_clim_embeddings)

In [ ]:
topic_term_matrix = topic_model.c_tf_idf_
words = topic_model.vectorizer_model.get_feature_names_out()

In [ ]:
topic_words

## Calculating Coherence

In [ ]:
documents = pd.DataFrame({"Document": filtered_text,
                          "ID": range(len(filtered_text)),
                          "Topic": topics[0]})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics[0]))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()

In [ ]:
coherence

In [ ]:
kmeans_model = KMeans(n_clusters = k, init = 'k-means++',random_state = 0, n_init = 10)

In [ ]:
topic_model = BERTopic(
    vectorizer_model = vectorizer_model,
    umap_model = umap_model,
    hdbscan_model = kmeans_model,
    language="english", 
    representation_model = representation_model, #diversify topic words
    calculate_probabilities=False, 
    verbose=True, 
    #nr_topics = 'auto'
    # min_topic_size = 50, 
    # n_gram_range = (1,2)
)
topics = topic_model.fit_transform(docs, embeddings)